In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are avai_lable in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'store.csv', 'sample_submission.csv']


In [2]:
! pip install pyspark

    100% |████████████████████████████████| 213.4MB 102kB/s eta 0:00:01
    100% |████████████████████████████████| 204kB 29.8MB/s ta 0:00:01
  Stored in directory: /tmp/.cache/pip/wheels/cd/54/c2/abfcc942eddeaa7101228ebd6127a30dbdf903c72db4235b23
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("task1").getOrCreate()

In [5]:
train_path = "../input/train.csv"

In [6]:
from pyspark.sql.functions import lag,date_format,udf
from pyspark.sql.types import IntegerType,ArrayType,TimestampType

In [7]:
df = spark.read.csv(train_path,inferSchema=True,header=True)
#df = df.filter("Store = 1")

In [8]:
from pyspark.sql import Window
from pyspark.sql.functions import lag,date_format,col,sum,weekofyear,year,mean,round,when,collect_list,dayofweek,expr

In [9]:
df = df.withColumn("year",year(df["date"]))

In [10]:
df = df.withColumn("week_id",weekofyear(df["date"]))

In [11]:
def date_range(min_date,max_date):
    base = min_date
    numdays =  ((max_date-min_date).days) +1
    date_list = []
    for x in range(0, numdays):
        date_list.append([base + timedelta(days=x)])
    return date_list

In [12]:
col_name="Date"
from pyspark.sql.functions import min,max,col
from datetime import datetime,timedelta
min_date = df.select(min(col(col_name))).collect()[0][0]
max_date = df.select(max(col(col_name))).collect()[0][0]
date_list = date_range(min_date,max_date)
full_date = spark.createDataFrame(date_list,[col_name])

## <center style='color:#e74c3c'>is date continuous</center>

## new imputation

In [13]:
store = df.select("store").distinct()

In [14]:
store_date = store.crossJoin(full_date)

In [15]:
missing_data = store_date.select("store","date").exceptAll(df.select("store","date"))

In [16]:
missing_data = missing_data.select("store",dayofweek("date").alias("DayOfWeek"),"date",expr("0").alias("sales"),expr("0").alias("customers"),expr("0").alias("open"),expr("-1").alias("promo"),expr("1").alias("StateHoliday"),expr("1").alias("SchoolHoliday"),year("date").alias("year"),weekofyear("date").alias("week_id"))

In [ ]:
#missing_data.count()

In [17]:
comp = df.union(missing_data)

In [18]:
close = df.filter("open=1").groupBy("year","week_id","store").agg(mean("sales").alias("avg_sales"),round(mean("Customers"),0).alias("avg_customers")).orderBy("store","year","week_id")


In [ ]:
#close.show()

In [19]:
new_df = comp.join(close,on=((comp.year==close.year) & (comp.week_id==close.week_id) & (comp.Store==close.store) ))
new_df = new_df.select(comp.Store,comp.DayOfWeek,comp.Date,when(comp.Sales==0,close.avg_sales).otherwise(comp.Sales).alias("sales"), when(comp.Customers==0,close.avg_customers).otherwise(comp.Customers).alias("customers"),comp.Promo,comp.StateHoliday,comp.SchoolHoliday,comp.year,comp.week_id ).orderBy("Store","year","week_id")

In [ ]:
#new_df.show()

new gdf

In [20]:
gdf = new_df.groupBy("year","week_id","store").agg(sum("sales").alias("weekly_sales")).orderBy("year","week_id","store")

In [ ]:
#gdf.show(2000)

In [21]:
win = Window.partitionBy("store").orderBy("year","week_id","store")
cuml_sales = gdf.withColumn("cuml_sales",sum("weekly_sales").over(win))
#cuml_sales.show()

In [33]:
cuml_sales.orderBy("store","year","week_id").show(1000)

+----+-------+-----+------------------+------------------+
|year|week_id|store|      weekly_sales|        cuml_sales|
+----+-------+-----+------------------+------------------+
|2013|      1|    1|38526.666666666664|38526.666666666664|
|2013|      2|    1|           38444.0| 76970.66666666666|
|2013|      3|    1|30307.666666666668|107278.33333333333|
|2013|      4|    1| 38582.83333333333|145861.16666666666|
|2013|      5|    1| 33475.16666666667| 179336.3333333333|
|2013|      6|    1| 41732.83333333333|221069.16666666663|
|2013|      7|    1|32526.666666666668|253595.83333333328|
|2013|      8|    1| 38442.83333333333| 292038.6666666666|
|2013|      9|    1|           31531.5| 323570.1666666666|
|2013|     10|    1| 43185.33333333333|366755.49999999994|
|2013|     11|    1|           32875.5|399630.99999999994|
|2013|     12|    1| 41441.16666666667| 441072.1666666666|
|2013|     13|    1|           48288.8| 489360.9666666666|
|2013|     14|    1|           33413.8| 522774.766666666

In [32]:
#new_df.filter("store=1 and week_id=2 and year=2013").select(sum("sales")).show()

+----------+
|sum(sales)|
+----------+
|   38444.0|
+----------+



In [30]:
#new_df.filter("store=1 and week_id=1 and year=2013").select("store","week_id","year","sales").show()

+-----+-------+----+-----------------+
|store|week_id|year|            sales|
+-----+-------+----+-----------------+
|    1|      1|2013|           2362.0|
|    1|      1|2013|           7193.0|
|    1|      1|2013|4815.833333333333|
|    1|      1|2013|           4997.0|
|    1|      1|2013|           4486.0|
|    1|      1|2013|           4327.0|
|    1|      1|2013|           5530.0|
|    1|      1|2013|4815.833333333333|
+-----+-------+----+-----------------+

